In [5]:

# ============================================================================
# 1. IMPORT THƯ VIỆN CẦN THIẾT
# ============================================================================


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import chi2_contingency, f_oneway
import warnings
warnings.filterwarnings('ignore')


# Thiết lập style cho visualization
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline


# Cấu hình hiển thị pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

print("✓ Đã import tất cả thư viện cần thiết")

✓ Đã import tất cả thư viện cần thiết


In [13]:
# ============================================================================
# 2. CLASS ĐỂ QUẢN LÝ QUY TRÌNH
# ============================================================================

class DataExplorationPipeline:
    """
    Class chính để xử lý và khám phá dữ liệu
    """


    def __init__(self, data_path=None, df=None):
        """
        Khởi tạo pipeline
        
        Parameters:
        -----------
        data_path : str, optional
            Đường dẫn đến file dữ liệu
        df : DataFrame, optional
            DataFrame có sẵn
        """

        if df is not None:
            self.df_original = df.copy()
        elif data_path:
            self.df_original = self._load_data(data_path)
        else:
            raise ValueError("Cần cung cấp data_path hoặc df")

        self.df = self.df_original.copy()
        self.numeric_cols = []
        self.categorical_cols = []
        self.datetime_cols = []
        self.text_cols = []
        self.report = {}

        print(f"✓ Đã load dữ liệu: {self.df.shape[0]} hàng, {self.df.shape[1]} cột")


    def _load_data(self, path):
        """Load dữ liệu từ nhiều định dạng"""

        if path.endswith('.csv'):
            return pd.read_csv(path)
        elif path.endswith('.xlsx') or path.endswith('.xls'):
            return pd.read_excel(path)
        elif path.endswith('.json'):
            return pd.read_json(path)
        elif path.endswith('.parquet'):
            return pd.read_parquet(path)
        else:
            raise ValueError("Định dạng file không được hỗ trợ")


pipeline = DataExplorationPipeline(data_path='./data/diabetes.csv')

✓ Đã load dữ liệu: 768 hàng, 9 cột


In [ ]:
# ========================================================================
# 3. PHÂN TÍCH TỔNG QUAN (INITIAL ASSESSMENT)
# ========================================================================

def initial_assessment(self):
    """
    Phân tích tổng quan ban đầu của tập dữ liệu
    """

    print("="*80)
    print("📋 PHÂN TÍCH TỔNG QUAN DỮ LIỆU")
    print("="*80)

    # ======== 3.1. Xem mẫu dữ liệu ========
    print("\n🔍 5 HÀNG ĐẦU TIÊN:")
    print(self.df.head())

    print("\n🔍 5 HÀNG CUỐI CÙNG:")
    print(self.df.tail())

    # ======== 3.2. Thông tin cơ bản ========
    print("\n📊 THÔNG TIN CƠ BẢN:")
    print(f"  • Số hàng: {self.df.shape[0]:,}")
    print(f"  • Số cột: {self.df.shape[1]}")
    print(f"  • Dung lượng bộ nhớ: {self.df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

    # ======== 3.3. Kiểu dữ liệu ========
    print("\n🏷️  KIỂU DỮ LIỆU:")
    dtype_counts = self.df.dtypes.value_counts()
    for dtype, count in dtype_counts.items():
        print(f"  • {dtype}: {count} cột")


    print("\n📝 CHI TIẾT CÁC CỘT:")
    info_df = pd.DataFrame({
        'Cột': self.df.columns,
        'Kiểu dữ liệu': self.df.dtypes.values,
        'Giá trị null': self.df.isnull().sum().values,
        'Tỷ lệ null (%)': (self.df.isnull().sum() / len(self.df) * 100).values,
        'Giá trị duy nhất': [self.df[col].nunique() for col in self.df.columns]
    })
    print(info_df.to_string(index=False))

    # ======== 3.4. Phân loại cột ========
    self._classify_columns()

    print(f"\n🔢 CÁC CỘT SỐ ({len(self.numeric_cols)}):")
    print(f"  {', '.join(self.numeric_cols)}")

    print(f"\n📦 CÁC CỘT PHÂN LOẠI ({len(self.categorical_cols)}):")
    print(f"  {', '.join(self.categorical_cols)}")

    if self.datetime_cols:
        print(f"\n📅 CÁC CỘT THỜI GIAN ({len(self.datetime_cols)}):")
        print(f"  {', '.join(self.datetime_cols)}")


    # ======== 3.5. Dữ liệu trùng lặp ========
    duplicates = self.df.duplicated().sum()
    print(f"\n🔄 DỮ LIỆU TRÙNG LẶP:")
    print(f"  • Số hàng trùng: {duplicates} ({duplicates/len(self.df)*100:.2f}%)")


    # Lưu report
    self.report['initial_assessment'] = {
        'shape': self.df.shape,
        'memory_mb': self.df.memory_usage(deep=True).sum() / 1024**2,
        'duplicates': duplicates,
        'column_types': {
            'numeric': len(self.numeric_cols),
            'categorical': len(self.categorical_cols),
            'datetime': len(self.datetime_cols)
        }
    }

    return info_df



def _classify_columns(self):
    """Phân loại các cột theo kiểu dữ liệu"""

    for col in self.df.columns:

        # Cột số
        if pd.api.types.is_numeric_dtype(self.df[col]):
            self.numeric_cols.append(col)

        # Cột datetime
        elif pd.api.types.is_datetime64_any_dtype(self.df[col]):
            self.datetime_cols.append(col)

        # Cột phân loại
        elif pd.api.types.is_categorical_dtype(self.df[col]) or pd.api.types.is_object_dtype(self.df[col]):
            # Kiểm tra xem có phải text dài không
            if self.df[col].dtype == 'object':
                avg_length = self.df[col].dropna().astype(str).str.len().mean()
                if avg_length > 50:  # Text dài
                    self.text_cols.append(col)
                else:
                    self.categorical_cols.append(col)
            else:
               self.categorical_cols.append(col)
 

info_df = pipeline.initial_assessment()


IndentationError: unexpected indent (1680882060.py, line 5)

In [ ]:
# ========================================================================
# 4. THỐNG KÊ MÔ TÃ (DESCRIPTIVE STATISTICS)
# ========================================================================

def description_statistics(self):
    """
    Thống kê mô tả chi tiết
    """

    print("\n" + "="*80)
    print("📈 THỐNG KÊ MÔ TẢ CHI TIẾT")
    print("="*80)

    # ======== 4.1. Thống kê cho biến số ========
    if self.numeric_cols:
        print("\n🔢 BIẾN SỐ:")
        desc_numeric = self.df[self.numeric_cols].describe(
            percentiles=[.01, .05, .25, .5, .75, .95, .99]
        ).T

        # Thêm thông tin bổ sung
        desc_numeric['missing'] = self.df[self.numeric_cols].isnull().sum()
        desc_numeric['missing_%'] = (desc_numeric['missing'] / len(self.df) * 100)
        desc_numeric['skewness'] = self.df[self.numeric_cols].skew()
        desc_numeric['kurtosis'] = self.df[self.numeric_cols].kurtosis()
        
        print(desc_numeric)


    # ======== 4.2. Thống kê cho biến phân loại ========
    if self.categorical_cols:
        print("\n📦 BIẾN PHÂN LOẠI:")
        for col in self.categorical_cols[:10]:  # Hiển thị tối đa 10 cột
            print(f"\n  → {col}:")
            value_counts = self.df[col].value_counts()
            missing = self.df[col].isnull().sum()

            print(f"     • Số giá trị duy nhất: {self.df[col].nunique()}")
            print(f"     • Giá trị thiếu: {missing} ({missing/len(self.df)*100:.2f}%)")
            print(f"     • Top 5 giá trị:")
            for val, count in value_counts.head().items():
                print(f"       - {val}: {count} ({count/len(self.df)*100:.2f}%)")

    return desc_numeric if self.numeric_cols else None


pipeline.descriptive_statistics()

In [ ]:
# ========================================================================
# 5. PHÂN TÍCH GIÁ TRỊ BỊ THIẾU (MISSING VALUES ANALYSIS)
# ========================================================================

def analyze_missing_values(self, plot=True):
    """
    Phân tích chi tiết giá trị bị thiếu
    
    Parameters:
    -----------
    plot : bool
        Có vẽ biểu đồ hay không
    """

    print("\n" + "="*80)
    print("🕳️  PHÂN TÍCH GIÁ TRỊ BỊ THIẾU")
    print("="*80)

    # Tính toán missing values
    missing_data = pd.DataFrame({
        'Column': self.df.columns,
        'Missing_Count': self.df.isnull().sum(),
        'Missing_Percentage': (self.df.isnull().sum() / len(self.df) * 100)
    })
    missing_data = missing_data[missing_data['Missing_Count'] > 0].sort_values(
        'Missing_Percentage', ascending=False
    )

    if len(missing_data) == 0:
        print("✓ Không có giá trị bị thiếu trong tập dữ liệu!")
        return None

    print(f"\n📊 Tổng quan:")
    print(f"  • Số cột có giá trị thiếu: {len(missing_data)}/{len(self.df.columns)}")
    print(f"  • Tổng số giá trị thiếu: {missing_data['Missing_Count'].sum():,}")
    print(f"  • Tỷ lệ dữ liệu thiếu: {(missing_data['Missing_Count'].sum() / (len(self.df) * len(self.df.columns)) * 100):.2f}%")
    
    print("\n📋 Chi tiết các cột:")
    print(missing_data.to_string(index=False))

    # Phân loại mức độ thiếu
    severe = missing_data[missing_data['Missing_Percentage'] > 50]
    moderate = missing_data[(missing_data['Missing_Percentage'] > 20) & 
                               (missing_data['Missing_Percentage'] <= 50)]
    mild = missing_data[missing_data['Missing_Percentage'] <= 20]

    print(f"\n🚨 Phân loại mức độ:")
    print(f"  • Nghiêm trọng (>50%): {len(severe)} cột")
    print(f"  • Trung bình (20-50%): {len(moderate)} cột")
    print(f"  • Nhẹ (<20%): {len(mild)} cột")

    # Visualization
    if plot and len(missing_data) > 0:
        fig, axes = plt.subplots(1, 2, figsize=(15, 5))
            
        # Bar plot
        missing_data.head(15).plot(
            x='Column', y='Missing_Percentage', kind='barh', 
            ax=axes[0], color='coral'
        )
        axes[0].set_title('Top 15 Cột Có Nhiều Giá Trị Thiếu Nhất', fontsize=12, fontweight='bold')
        axes[0].set_xlabel('Tỷ lệ thiếu (%)')
        axes[0].set_ylabel('')
        
        # Heatmap của missing pattern
        missing_cols = missing_data.head(15)['Column'].tolist()
        if len(missing_cols) > 0:
            sns.heatmap(
                self.df[missing_cols].isnull(), 
                cbar=False, 
                yticklabels=False,
                cmap='viridis',
                ax=axes[1]
            )
            axes[1].set_title('Pattern của Giá Trị Thiếu', fontsize=12, fontweight='bold')
        
        plt.tight_layout()
        plt.show()
    
    self.report['missing_values'] = missing_data.to_dict('records')
    return missing_data


pipeline.analyze_missing_values(plot=True)

In [ ]:
# ========================================================================
# 6. TRỰC QUAN HÓA PHÂN PHỐI (DISTRIBUTION VISUALIZATION)
# ========================================================================

def visualize_distributions(self, max_cols=10):
    """
    Trực quan hóa phân phối của các biến
    
    Parameters:
    -----------
    max_cols : int
        Số lượng cột tối đa để hiển thị
    """

    print("\n" + "="*80)
    print("📊 TRỰC QUAN HÓA PHÂN PHỐI")
    print("="*80)

    # ======== 6.1. Biến số ========
    if self.numeric_cols:
        print("\n🔢 Phân phối biến số:")

        cols_to_plot = self.numeric_cols[:max_cols]
        n_cols = min(3, len(cols_to_plot))
        n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

        fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 4*n_rows))
        axes = axes.flatten() if n_rows > 1 or n_cols > 1 else [axes]

        for idx, col in enumerate(cols_to_plot):
            data = self.df[col].dropna()

            # Histogram + KDE
            axes[idx].hist(data, bins=50, alpha=0.6, color='skyblue', edgecolor='black', density=True)

            # KDE
            if len(data) > 1:
                data.plot(kind='kde', ax=axes[idx], color='red', linewidth=2)

            axes[idx].set_title(f'{col}\n(Skew: {data.skew():.2f}, Kurt: {data.kurtosis():.2f})', fontsize=10)
            axes[idx].set_xlabel('')
            axes[idx].grid(True, alpha=0.3)

        # Ẩn các subplot thừa
        for idx in range(len(cols_to_plot), len(axes)):
            axes[idx].axis('off')

        plt.tight_layout()
        plt.show()

    
    # ======== 6.2. Biến phân loại ========
    if self.categorical_cols:
        print("\n📦 Phân phối biến phân loại:")

        cols_to_plot = [col for col in self.categorical_cols[:max_cols] 
                           if self.df[col].nunique() <= 20]

        if cols_to_plot:
            n_cols = min(3, len(cols_to_plot))
            n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

            fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 4*n_rows))
            axes = axes.flatten() if n_rows > 1 or n_cols > 1 else [axes]

            for idx, col in enumerate(cols_to_plot):
                value_counts = self.df[col].value_counts().head(10)

                axes[idx].barh(range(len(value_counts)), value_counts.values, color='teal')
                axes[idx].set_yticks(range(len(value_counts)))
                axes[idx].set_yticklabels(value_counts.index)
                axes[idx].set_title(f'{col} (Top 10)', fontsize=10)
                axes[idx].set_xlabel('Số lượng')
                axes[idx].grid(True, alpha=0.3, axis='x')

        for idx in range(len(cols_to_plot), len(axes)):
            axes[idx].axis('off')
            
        plt.tight_layout()
        plt.show()


pipeline.visualize_distributions(max_cols=12)

In [ ]:
# ============================================================================
# 7. KHỞI TẠO VÀ SỬ DỤNG
# ============================================================================

"""
HƯỚNG DẪN SỬ DỤNG:

# Bước 1: Load dữ liệu
pipeline = DataExplorationPipeline(data_path='your_data.csv')
# Hoặc: pipeline = DataExplorationPipeline(df=your_dataframe)

# Bước 2: Chạy phân tích ban đầu
pipeline.initial_assessment()

# Bước 3: Thống kê mô tả
pipeline.descriptive_statistics()

# Bước 4: Phân tích missing values
pipeline.analyze_missing_values(plot=True)

# Bước 5: Visualize phân phối
pipeline.visualize_distributions(max_cols=12)
"""

print("\n✅ ĐÃ HOÀN THÀNH PHẦN 1: SETUP & EDA CƠ BẢN")
print("📌 Tiếp theo: Chạy Part 2 để xử lý Data Cleaning & Preprocessing")